In [1]:
%matplotlib inline

import os,sys

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import torch
from torch.autograd import Variable

import timeit

%load_ext autoreload
%autoreload 2

# Loading the files for training

In [2]:
# Directory and files name
train_dir = "training/"
tr_image_dir = train_dir + "images/"
tr_label_dir = train_dir + "groundtruth/"

tr_image_files = os.listdir(tr_image_dir)
tr_label_files = os.listdir(tr_label_dir)

# Number of training samples
N = len(tr_image_files)

# Load the images and ground truth
img_train = []
label_train = []
for i in range(N):
    img = mpimg.imread(tr_image_dir + tr_image_files[i])
    label = mpimg.imread(tr_label_dir + tr_label_files[i])
    
    img_train.append(img)
    label_train.append(label)

# Keep only sub-set of images
NUM_IMAGES = N

img_train = np.asarray(img_train[:NUM_IMAGES])
label_train = np.asarray(label_train[:NUM_IMAGES])

print(img_train.shape, label_train.shape)

(100, 400, 400, 3) (100, 400, 400)


In [4]:
PATCH_SIZE = 16

In [5]:
def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

label_patches = [img_crop(label_train[i, :, :], PATCH_SIZE, PATCH_SIZE) for iNUM_IMAGES in range(NUM_IMAGES)]
label_train =  np.asarray([label_patches[i][j] for i in range(len(label_patches)) for j in range(len(label_patches[i]))])

In [13]:
# Compute features for each image patch
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0

label_train = np.asarray([value_to_class(np.mean(label_train[i])) for i in range(len(label_train))])

# Enhance the training dataset w/ sliding windows

In [6]:
IMG_SIZE = 400
WINDOW_SIZE = 52 # 18px - PATCH_SIZE - 18px
NB_WINDOWS = (IMG_SIZE/PATCH_SIZE)**2

In [7]:
def apply_mirror_boundary_conditions(coord, dim):
    """
    Return the correct coordinate according to mirror boundary conditions
        coord: a coordinate (x or y) in the image
        dim: the length of the axis of said coordinate
    """
    # If the coordinate is outside of the bounds of the axis, take its reflection inside the image
    if coord < 0:
        coord = -coord
    elif coord >= dim:
        coord =  2*(dim-1) - coord % (2*(dim-1))
    # Else, do nothing
    return int(coord)

def get_window(image, window_size, corner_coordinates, patch_size):
    """
    Get a window in image, centered on a patch, taking into account boundary conditions
        image: a numpy array representing our image
        window_size: an even number specifying the size of the window
        corner_coordinates: a list containing the x-y coordinates of the patch's upleft pixel
        path_size: an even number specifying the size of the central patch
    """
    # Get convenient variables
    window_radius = window_size/2
    border_size = (window_size - patch_size)/2
    i_patch_corner, j_patch_corner = (corner_coordinates[0], corner_coordinates[1])
    i_window_corner, j_window_corner = (i_patch_corner - border_size, j_patch_corner - border_size)
    nrows, ncols, nchannels = image.shape
    window = np.zeros((window_size, window_size, nchannels))
    
    # Fill in the window array with pixels of the image
    for i in range(window_size):
        # Apply mirror boundary conditions on the x-coordinate
        i_mirrored = apply_mirror_boundary_conditions(i_window_corner + i, nrows)
        for j in range(window_size):
            # Same for the y-coordinate
            j_mirrored = apply_mirror_boundary_conditions(j_window_corner + j, ncols)
            # Fill in the window with the corresponding pixel
            window[i, j, :] = image[i_mirrored, j_mirrored, :]
    return window

In [8]:
def shift_to_the_right(image, window, corner_coordinates, patch_size):
    nrows, ncols, _ = image.shape
    window_size = len(window)
    #window_radius = window_size/2
    border_size = (window_size - patch_size)/2
    i_patch_corner, j_patch_corner = (corner_coordinates[0], corner_coordinates[1])
    i_window_corner, j_window_corner = (i_patch_corner - border_size, j_patch_corner - border_size)
    step = patch_size
    
    shifted = np.roll(window, -step, axis=1)
    for i in range(window_size):
        i_mirrored = apply_mirror_boundary_conditions(i_window_corner + i, nrows)            
        for j in range(window_size-step, window_size):
            j_mirrored = apply_mirror_boundary_conditions(j_window_corner + j + step, ncols)
            shifted[i, j, :] = image[i_mirrored, j_mirrored, :]
    return shifted

def shift_to_the_bottom(image, window, corner_coordinates, patch_size):
    nrows, ncols, _ = image.shape
    window_size = len(window)
    #window_radius = window_size/2
    border_size = (window_size - patch_size)/2
    i_patch_corner, j_patch_corner = (corner_coordinates[0], corner_coordinates[1])
    i_window_corner, j_window_corner = (i_patch_corner - border_size, j_patch_corner - border_size)
    step = patch_size
    
    shifted = np.roll(window, -step, axis=0)
    for j in range(window_size):
        j_mirrored = apply_mirror_boundary_conditions(j_window_corner + j, ncols)
        for i in range(window_size-step, window_size):
            i_mirrored = apply_mirror_boundary_conditions(i_window_corner + i + step, nrows)
            shifted[i, j, :] = image[i_mirrored, j_mirrored, :]
    return shifted

def sliding_window(image, window_size, patch_size):
    """
    Construct a list of sliding windows of given size on an image.
    The windows, centered on a patch, will slide from left to right and from up to down.
        image: a numpy array representing our image
        window_size: an even number specifying the size of the window
        patch_size: the size of the central patch
    """
    nrows, ncols, _ = image.shape
    windows = []
    i = 0
    row_windows = [get_window(image, window_size, [i, 0], patch_size)]
    for j in range(patch_size, ncols-1, patch_size):
        #print(j)
        row_windows += [shift_to_the_right(image, row_windows[-1], [i, j], patch_size)]
    windows += row_windows
    #print('===')
    for i in range(patch_size, nrows-1, patch_size):
        #print(i)
        row_windows = [shift_to_the_bottom(image, row_windows[int(j/patch_size)], [i, j], patch_size) 
                       for j in range(0, ncols-1, patch_size)]
        #print(len(row_windows))
        windows += row_windows
    return windows

In [10]:
def compute(img_train, window_size, patch_size):
    train_data = []
    #train_labels = []
    for im, i in zip(img_train, range(len(img_train))):
        w_im = sliding_window(im, window_size, patch_size)
        #w_labels = sliding_window(labels[:, :, np.newaxis], window_size, step)
        train_data += w_im
        #train_labels += w_labels
        path = './windows_train_patch/' + str(i)
        os.makedirs(path, exist_ok=True)
        for wi, j in zip(w_im, range(len(w_im))):
            img_name = path + '/im_' + str(j) + '.png'
            plt.imsave(img_name, wi)
            #label_name = path + '/label_' + str(j) + '.png'
            #plt.imsave(label_name, wl[:,:,0])
    return train_data

In [11]:
img_train = np.asarray(compute(img_train, WINDOW_SIZE, PATCH_SIZE))

In [ ]:
#a = np.array(range(10*10))
#a = a.reshape((10, 10))
#print(a)
#aa = get_window(a[:,:,np.newaxis], 5, [5,5])[:,:,0]
#print('\n')
#print(aa)
#print('\n')
#print(shift_to_the_bottom(a[:,:,np.newaxis], aa[:,:,np.newaxis], [5,5], 2)[:,:,0])

# Neural networks

In [26]:
NUM_CHANNELS = 3 # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
IMG_PATCH_SIZE = 52
TRAINING_SIZE = 62500
BATCH_SIZE = 32
NUM_EPOCHS = 15
a = 0.00

In [17]:
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, LeakyReLU, Input, Reshape, Permute, Average
from keras.layers import Conv2D, MaxPooling2D, Concatenate, Lambda, Activation, GlobalAveragePooling2D, Conv2DTranspose, GlobalAveragePooling1D
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


In [27]:
model = Sequential()
model.add(Conv2D(32, 2, input_shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS)))
model.add(LeakyReLU(alpha=a))
model.add(Conv2D(32, 2))
model.add(LeakyReLU(alpha=a))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, 2))
model.add(LeakyReLU(alpha=a))
model.add(Conv2D(64, 2))
model.add(LeakyReLU(alpha=a))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Dropout(0.5))
model.add(LeakyReLU(alpha=a))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
from keras.engine.topology import Layer
def fire(x, squeeze, expand):
    x = Conv2D(squeeze, 1)(x)
    x = LeakyReLU(alpha=a)(x)
    e11 = Conv2D(expand, 1)(x)
    e11 = LeakyReLU(alpha=a)(e11)
    e33 = Conv2D(expand, 1)(x)
    e33 = LeakyReLU(alpha=a)(e33)
    return Concatenate(axis=3)([e11, e33])

In [ ]:
inputs = Input(shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS))
x = Conv2D(32, kernel_size=3, strides=2, input_shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS))(inputs)
x = LeakyReLU(alpha=a)(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

x = fire(x, squeeze=8, expand=16)
x = fire(x, squeeze=8, expand=16)
x = MaxPooling2D(pool_size=3, strides=2)(x)

'''x = fire(x, squeeze=32, expand=128)
x = fire(x, squeeze=32, expand=128)
x = MaxPooling2D(pool_size=3, strides=2)(x)

x = fire(x, squeeze=48, expand=192)
x = fire(x, squeeze=48, expand=192)
x = fire(x, squeeze=64, expand=256)
x = fire(x, squeeze=64, expand=256)
x = MaxPooling2D(pool_size=3, strides=2)(x)'''
x = Flatten()(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs, x)

In [22]:
model = Sequential()
model.add(Conv2D(32, 2, input_shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS)))
model.add(LeakyReLU(alpha=a))
model.add(Conv2D(32, 2))
model.add(LeakyReLU(alpha=a))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, 2))
model.add(LeakyReLU(alpha=a))
model.add(Conv2D(64, 2))
model.add(LeakyReLU(alpha=a))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(2, 1))
model.add(Conv2DTranspose(1, kernel_size=IMG_PATCH_SIZE - 1, padding='valid'))
#model.add(Reshape((-1, IMG_PATCH_SIZE * IMG_PATCH_SIZE)))
#model.add(Permute((2,1)))
model.add(GlobalAveragePooling2D())
model.add(Activation('sigmoid'))

In [19]:
for layer in model.layers:
    print(layer.input_shape,layer.output_shape )

(None, 51, 51, 3) (None, 50, 50, 32)
(None, 50, 50, 32) (None, 50, 50, 32)
(None, 50, 50, 32) (None, 49, 49, 32)
(None, 49, 49, 32) (None, 49, 49, 32)
(None, 49, 49, 32) (None, 24, 24, 32)
(None, 24, 24, 32) (None, 24, 24, 32)
(None, 24, 24, 32) (None, 23, 23, 64)
(None, 23, 23, 64) (None, 23, 23, 64)
(None, 23, 23, 64) (None, 22, 22, 64)
(None, 22, 22, 64) (None, 22, 22, 64)
(None, 22, 22, 64) (None, 11, 11, 64)
(None, 11, 11, 64) (None, 11, 11, 64)
(None, 11, 11, 64) (None, 7744)
(None, 7744) (None, 128)
(None, 128) (None, 128)
(None, 128) (None, 128)
(None, 128) (None, 1)


In [ ]:
LR = 0.001
DECAY = 0.00000
adam = Adam(lr=LR, decay=DECAY)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])

model.fit(img_train, label_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS)

Epoch 1/15
 3392/62500 [>.............................] - ETA: 35:12 - loss: 0.6301 - acc: 0.7005

In [ ]:
score = model.evaluate(train_data, train_labels)
score